<a href="https://colab.research.google.com/github/minhaz1172/Deep-Learning/blob/main/keras_Functional_Api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# california housing datasets to predict by functional api

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# load the dataset

In [2]:
data=fetch_california_housing()
df=pd.DataFrame(data.data,columns=data.feature_names)
df['Target']=data.target #data.target values were already part of the full dataset
print(df.head())



   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  Target  
0    -122.23   4.526  
1    -122.22   3.585  
2    -122.24   3.521  
3    -122.25   3.413  
4    -122.25   3.422  


# X has shape (n_samples, 8)    AND  For demonstration, split the data into two groups of features:
# Group 1: first 4 features, Group 2: last 4 features.

In [3]:
X,y=data.data,data.target

input1=X[:,:4]
input2=X[:,4:]
print(input1.shape,input2.shape)

(20640, 4) (20640, 4)


**# input1=scaler1.fit_transform(input1)**
standardizing the inputs.

👉 What it does:

Subtracts the mean from each feature

Divides by standard deviation

Result: Each feature has mean ≈ 0 and standard deviation ≈ 1

👉 Why?

Many ML models (e.g., linear regression, SVM, neural networks) perform better when input features are scaled.


# **y = y.reshape(-1, 1)**
#  y from a 1D array (like [2.5, 3.6, ...]) to a 2D array (like [[2.5], [3.6], ...]).

👉 Why?

StandardScaler expects 2D input.

Machine learning pipelines often need target arrays in 2D for consistency during preprocessing.

#SCALE BOTH INPUTS and target variable

In [5]:
scaler1=StandardScaler()
scaler2=StandardScaler()
input1=scaler1.fit_transform(input1)
input2= scaler2.fit_transform(input2)

y=y.reshape(-1,1) # reshape target to 2D for scaling
scale_y=StandardScaler()
y=scale_y.fit_transform(y)

# # Split the data into training and test sets (80-20 split)

In [6]:
X1_train,X1_test,X2_train,X2_test,y_train,y_test=train_test_split(input1,input2,y,test_size=0.2,random_state=42)

# building the model

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input,Dense,concatenate
from tensorflow.keras.models import Model

#define two layers
input_A=Input(shape=(4,),name="Input_A")
input_B=Input(shape=(4,),name="Input_B")

#Branch for input A
x1=Dense(32,activation='relu')(input_A)
x1=Dense(16,activation='relu')(x1)

# Branch for input_B
x2=Dense(32,activation='relu')(input_B)
x2=Dense(16,activation='relu')(x2)

combined=concatenate([x1,x2],name="concatenated layer")

# Add subsequent layers on the combined representation

z=Dense(32,activation='relu')(combined)
output=Dense(1,activation='linear',name="output")(z) #The final layer has 1 neuron with a linear activation, suitable for regression tasks

# building the model
model=Model(inputs=[input_A,input_B],outputs=output)

#display model summary
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input_A             │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Input_B             │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 32)        │        160 │ Input_A[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 32)        │        160 │ Input_B[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 16)        │        528 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 16)        │        528 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenated layer  │ (None, 32)        │          0 │ dense_1[0][0],    │
│ (Concatenate)       │                   │            │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 32)        │      1,056 │ concatenated      │
│                     │                   │            │ layer[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │         33 │ dense_4[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,465 (9.63 KB)

 Trainable params: 2,465 (9.63 KB)

 Non-trainable params: 0 (0.00 B)